# <center>Heat Equation</center>

In [ ]:
%%writefile modules/io.py

import numpy
import matplotlib.pyplot


def initialise_field(filename):
    field = numpy.loadtxt(filename)
    return field


def write_field(field, step):
    matplotlib.pyplot.gca().clear()
    matplotlib.pyplot.imshow(field)
    matplotlib.pyplot.axis('off')
    matplotlib.pyplot.savefig('output/bottle_{0:04d}.png'.format(step))


def to_text_file(field, filename):
    numpy.savetxt(filename, field, fmt='%1.1f')

In [ ]:
%%writefile modules/evolve.py

def evolve(u, u_previous, a, dt, dx2, dy2):

    del_sqrd_u = (u_previous[:-2, 1:-1] - 2 * u_previous[1:-1, 1:-1] +
                  u_previous[2:, 1:-1]) / dx2 + (u_previous[1:-1, :-2] -
                                                 2 * u_previous[1:-1, 1:-1] +
                                                 u_previous[1:-1, 2:]) / dy2

    u[1:-1, 1:-1] = u_previous[1:-1, 1:-1] + dt * a * del_sqrd_u

    u_previous[:, :] = u[:, :]

In [ ]:
%%writefile modules/comms.py

from mpi4py import MPI
import numpy

def halo_exchange(field, rank, num_processes):
    # determine neighbours
    up = (rank - 1) % num_processes
    down = (rank + 1) % num_processes
    
    # checkerboard scheme to avoid deadlocks
    if (rank % 2) == 0:
        # send bottom border down
        MPI.COMM_WORLD.Send(field[-2], down)
        # send top border up
        MPI.COMM_WORLD.Send(field[1], up)
        # receive down neighbour's border to down halo region
        MPI.COMM_WORLD.Recv(field[-1], down)
        # receive up neighbour's border to up halo region
        MPI.COMM_WORLD.Recv(field[0], up)
    else:
        # match alternating communications
        MPI.COMM_WORLD.Recv(field[0], up)
        MPI.COMM_WORLD.Recv(field[-1], down)
        MPI.COMM_WORLD.Send(field[1], up)
        MPI.COMM_WORLD.Send(field[-2], down)
        

def distribute_subfields(field, rank, num_processes):
    dims = numpy.empty(2, dtype=int)
    if rank == 0:
        num_rows, num_cols = field.shape

        num_subfield_rows = num_rows // num_processes
        dims[0] = num_subfield_rows + 2 # add halo regions
        dims[1] = num_cols
        
        for i in range(1, num_processes):
            MPI.COMM_WORLD.Send(dims, i)
    else:
        MPI.COMM_WORLD.Recv(dims, 0)

    local_field = numpy.zeros(dims)
    if rank == 0:
        local_field[1:-1] = field[0:(dims[0] - 2)]
        for i in range(1, num_processes):
            MPI.COMM_WORLD.Send(field[i*(dims[0] - 2):(i + 1)*(dims[0] - 2)], i)
    else:
        MPI.COMM_WORLD.Recv(local_field[1:-1], 0)

    return local_field


def gather_subfields(local_field, rank, num_processes):
    dims = numpy.empty(2, dtype=int)
    dims[0], dims[1] = local_field[1:-1].shape
    dims[0] *= num_processes
    field = numpy.empty(dims)
    
    if rank == 0:
        field[:dims[0]//num_processes] = local_field[1:-1]
        for i in range(1, num_processes):
            MPI.COMM_WORLD.Recv(field[i*dims[0]//num_processes:(i + 1)*dims[0]//num_processes], i)
    else:
        MPI.COMM_WORLD.Send(local_field[1:-1], 0)

    return field

In [ ]:
%%writefile main.py

import matplotlib.pyplot
import time
import numpy
from mpi4py import MPI
from mpi4py.MPI import Request
from modules import evolve, io, comms

# Set the colormap
matplotlib.pyplot.rcParams['image.cmap'] = 'BrBG'

# Basic parameters
a = 0.5  # Diffusion constant
timesteps = 1000  # Number of time-steps to evolve system
image_interval = 4000  # Write frequency for png files

# Grid spacings
dx = 0.01
dy = 0.01
dx2 = dx**2
dy2 = dy**2

# For stability, this is the largest possible size of the time-step:
dt = dx2 * dy2 / (2 * a * (dx2 + dy2))


rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()


def iterate(field, field0, timesteps, image_interval):
    for i in range(1, timesteps + 1):
        comms.halo_exchange(field, rank, size)
        evolve.evolve(field, field0, a, dt, dx2, dy2)


def main():
    field = None
    if rank == 0:
        print('Initialising field...')
        field = io.initialise_field('data/bottle.dat')
    
        io.write_field(field, 0)
        print('Initial field written to output/bottle_0000.png.')

    local_field = comms.distribute_subfields(field, rank, size)
        
    local_field0 = local_field[:, :]

    if rank == 0:
        print('Simulating...')
        
    t0 = time.time()
    iterate(local_field, local_field0, timesteps, image_interval)
    t1 = time.time()
    
    if rank == 0:
        field = comms.gather_subfields(local_field, rank, size)
        print('Simulation complete.')
        io.write_field(field, timesteps)
        print('Final field written to output/bottle_{}.png.'.format(timesteps))
        print("Running Time: {0}".format(t1 - t0))
    else:
        comms.gather_subfields(local_field, rank, size)

main()

In [ ]:
!mkdir -p output

In [ ]:
!mpirun --oversubscribe -np 4 python3 main.py

In [ ]:
import matplotlib.image as img
import matplotlib.pyplot as plt

%matplotlib inline

initial_bottle = img.imread('output/bottle_0000.png')
final_bottle = img.imread('output/bottle_1000.png')

fig=plt.figure(figsize=(16, 16))

fig.add_subplot(1, 2, 1)
plt.gca().set_title('Initial Configuration')
plt.imshow(initial_bottle)
fig.add_subplot(1, 2, 2)
plt.gca().set_title('Final Configuration')
plt.imshow(final_bottle)

plt.show()